In [40]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import zoopla_clean as zc
import zoopla_scrape as zs

import importlib
importlib.reload(zc)
importlib.reload(zs)

to_scrape_df = pd.read_csv("to_scrape.csv")

to_scrape_df



,location,link
0,manchester,https://www.zoopla.co.uk/for-sale/property/man...
1,salford,https://www.zoopla.co.uk/for-sale/property/gre...
2,bolton,https://www.zoopla.co.uk/for-sale/property/gre...
3,wigan,https://www.zoopla.co.uk/for-sale/property/wig...
4,stockport,https://www.zoopla.co.uk/for-sale/property/sto...
5,bury,https://www.zoopla.co.uk/for-sale/property/gre...
6,rochdale,https://www.zoopla.co.uk/for-sale/property/roc...
7,oldham,https://www.zoopla.co.uk/for-sale/property/old...
8,tameside,https://www.zoopla.co.uk/for-sale/property/tam...
9,lancashire,https://www.zoopla.co.uk/for-sale/property/lan...


In [2]:
df = pd.DataFrame()
data_frames = []

urls = to_scrape_df['link'].to_list()

for url in urls:
    data_frames.append(zs.get_house_data(url))

Error count: 17
Error count: 3
Error count: 4
Error count: 0
Error count: 1
Error count: 0
Error count: 2
Error count: 5
Error count: 1
Error count: 71
Error count: 66
Error count: 50
Error count: 56


In [41]:
df_total = pd.concat(data_frames)
df = df_total.copy()

size_start = (len(df))
df = zc.clean_type(df)
size_end = (len(df))

print(f"Database of {size_start} reduced to {size_end} after cleaning and removing duplicates")
df = zc.clean_price(df)
size_end = (len(df))

print(f"Database of {size_start} reduced to {size_end} after cleaning and removing duplicates")
df = zc.get_dist_to_station(df)
size_end = (len(df))

print(f"Database of {size_start} reduced to {size_end} after cleaning and removing duplicates")
df = zc.clean_address(df)
size_end = (len(df))

print(f"Database of {size_start} reduced to {size_end} after cleaning and removing duplicates")
df = zc.remove_outliers(df)
size_end = (len(df))

print(f"Database of {size_start} reduced to {size_end} after cleaning and removing duplicates")
df = zc.drop_duplicates(df)
size_end = (len(df))

print(f"Database of {size_start} reduced to {size_end} after cleaning and removing duplicates")
df.head(4)

Database of 15574 reduced to 12620 after cleaning and removing duplicates
Database of 15574 reduced to 12568 after cleaning and removing duplicates
Database of 15574 reduced to 12568 after cleaning and removing duplicates
Database of 15574 reduced to 12525 after cleaning and removing duplicates
Database of 15574 reduced to 6397 after cleaning and removing duplicates
Database of 15574 reduced to 6397 after cleaning and removing duplicates


,type,price,station,beds,baths,receptions,post_code,location
158,flat,110000.0,1.3,2,1.0,1.0,M8,manchester
166,flat,110000.0,0.6,2,1.0,1.0,M40,manchester
168,flat,110000.0,0.5,1,1.0,1.0,M5,salford
169,flat,110000.0,NaN,2,1.0,1.0,M27,manchester


In [42]:
average_list = zs.get_post_code_data(df)
average_list

222 unique post codes
Error - 
Error - L18.


,post_code,avg_sold_price_12months,detached_12months,semi_detached_12months,terraced_12months,flat_12months
0,BB2,163524,285219,170276,102069,99833
1,M41,308916,441518,306433,257713,171913
2,L30,124747,217666,137166,100470,76750
3,L3,179201,62661,179687,187165,182574
4,BL1,205890,433943,210748,120851,120456
...,...,...,...,...,...,...
215,WA4,334261,524883,296558,226601,143500
216,FY6,232324,332794,188108,160839,117831
217,L5,101419,0,129423,96090,69207
218,PR6,221070,311319,208686,156358,157100


In [43]:
average_list.to_csv("post_code_list.csv",index=False)

In [66]:
df2 = pd.merge(df,average_list,left_on='post_code',right_on='post_code')
df2 = df2[df2['avg_sold_price_12months'].notna()]
df2 = df2.reset_index(drop=True)
df2

,type,price,station,beds,baths,receptions,post_code,location,avg_sold_price_12months,detached_12months,semi_detached_12months,terraced_12months,flat_12months
0,flat,110000.0,1.3,2,1.0,1.0,M8,manchester,162788,292000,208116,131410,101576
1,flat,115000.0,1.4,2,1.0,1.0,M8,manchester,162788,292000,208116,131410,101576
2,flat,170000.0,0.7,2,1.0,1.0,M8,manchester,162788,292000,208116,131410,101576
3,semi-detached,180000.0,1.4,3,1.0,1.0,M8,manchester,162788,292000,208116,131410,101576
4,semi-detached,190000.0,1.9,3,1.0,0.0,M8,manchester,162788,292000,208116,131410,101576
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6382,detached,600000.0,4.1,4,0.0,0.0,CA4,cumbria,272071,334295,245491,190861,166281
6383,detached,375000.0,1.7,4,1.0,1.0,CA4,carlisle,272071,334295,245491,190861,166281
6384,detached,447500.0,0.4,4,0.0,0.0,CA4,carlisle,272071,334295,245491,190861,166281
6385,detached,350000.0,1.4,5,0.0,0.0,CA4,carlisle,272071,334295,245491,190861,166281


In [67]:
df2.loc[df2['type'] == 'flat', 'avg_type'] = df2['flat_12months']
df2.loc[df2['type'] == 'detached', 'avg_type'] = df2['detached_12months']
df2.loc[df2['type'] == 'semi-detached', 'avg_type'] = df2['semi_detached_12months']
df2.loc[df2['type'] == 'terraced', 'avg_type'] = df2['terraced_12months']

In [68]:
df2

,type,price,station,beds,baths,receptions,post_code,location,avg_sold_price_12months,detached_12months,semi_detached_12months,terraced_12months,flat_12months,avg_type
0,flat,110000.0,1.3,2,1.0,1.0,M8,manchester,162788,292000,208116,131410,101576,101576.0
1,flat,115000.0,1.4,2,1.0,1.0,M8,manchester,162788,292000,208116,131410,101576,101576.0
2,flat,170000.0,0.7,2,1.0,1.0,M8,manchester,162788,292000,208116,131410,101576,101576.0
3,semi-detached,180000.0,1.4,3,1.0,1.0,M8,manchester,162788,292000,208116,131410,101576,208116.0
4,semi-detached,190000.0,1.9,3,1.0,0.0,M8,manchester,162788,292000,208116,131410,101576,208116.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6382,detached,600000.0,4.1,4,0.0,0.0,CA4,cumbria,272071,334295,245491,190861,166281,334295.0
6383,detached,375000.0,1.7,4,1.0,1.0,CA4,carlisle,272071,334295,245491,190861,166281,334295.0
6384,detached,447500.0,0.4,4,0.0,0.0,CA4,carlisle,272071,334295,245491,190861,166281,334295.0
6385,detached,350000.0,1.4,5,0.0,0.0,CA4,carlisle,272071,334295,245491,190861,166281,334295.0


In [69]:
df2 = df2.drop(columns=['detached_12months', 'semi_detached_12months', 'terraced_12months', 'flat_12months'])

In [70]:
df2

,type,price,station,beds,baths,receptions,post_code,location,avg_sold_price_12months,avg_type
0,flat,110000.0,1.3,2,1.0,1.0,M8,manchester,162788,101576.0
1,flat,115000.0,1.4,2,1.0,1.0,M8,manchester,162788,101576.0
2,flat,170000.0,0.7,2,1.0,1.0,M8,manchester,162788,101576.0
3,semi-detached,180000.0,1.4,3,1.0,1.0,M8,manchester,162788,208116.0
4,semi-detached,190000.0,1.9,3,1.0,0.0,M8,manchester,162788,208116.0
...,...,...,...,...,...,...,...,...,...,...
6382,detached,600000.0,4.1,4,0.0,0.0,CA4,cumbria,272071,334295.0
6383,detached,375000.0,1.7,4,1.0,1.0,CA4,carlisle,272071,334295.0
6384,detached,447500.0,0.4,4,0.0,0.0,CA4,carlisle,272071,334295.0
6385,detached,350000.0,1.4,5,0.0,0.0,CA4,carlisle,272071,334295.0


In [71]:
df2.isnull().sum()

type                         0
price                        0
station                    998
beds                         0
baths                        0
receptions                   0
post_code                    0
location                     0
avg_sold_price_12months      0
avg_type                     0
dtype: int64

In [72]:
df2['station'] = df2['station'].fillna(df2.groupby('post_code')['station'].transform('mean'))
df2 = df2.dropna()

In [73]:
df2.isnull().sum()

type                       0
price                      0
station                    0
beds                       0
baths                      0
receptions                 0
post_code                  0
location                   0
avg_sold_price_12months    0
avg_type                   0
dtype: int64

In [74]:
df2 = df2.reset_index(drop=True)
df2

,type,price,station,beds,baths,receptions,post_code,location,avg_sold_price_12months,avg_type
0,flat,110000.0,1.3,2,1.0,1.0,M8,manchester,162788,101576.0
1,flat,115000.0,1.4,2,1.0,1.0,M8,manchester,162788,101576.0
2,flat,170000.0,0.7,2,1.0,1.0,M8,manchester,162788,101576.0
3,semi-detached,180000.0,1.4,3,1.0,1.0,M8,manchester,162788,208116.0
4,semi-detached,190000.0,1.9,3,1.0,0.0,M8,manchester,162788,208116.0
...,...,...,...,...,...,...,...,...,...,...
6134,detached,600000.0,4.1,4,0.0,0.0,CA4,cumbria,272071,334295.0
6135,detached,375000.0,1.7,4,1.0,1.0,CA4,carlisle,272071,334295.0
6136,detached,447500.0,0.4,4,0.0,0.0,CA4,carlisle,272071,334295.0
6137,detached,350000.0,1.4,5,0.0,0.0,CA4,carlisle,272071,334295.0


In [75]:
df2.to_csv("zoopla_data.csv",index=False)

In [76]:
#4376